In [ ]:
from __future__ import print_function
from traitlets import Integer, Unicode, Bool, validate, TraitError
from ipywidgets import widgets, DOMWidget, register
import numpy as np

from traittypes import Array

In [ ]:
import ipywidgets as widgets
import IPython

from traitlets import Unicode, validate, CInt, CFloat
from traittypes import Array

import logging
import time
import numpy as np
import math

try:
  from io import BytesIO as StringIO # python3
except:
  from StringIO import StringIO # python2

from base64 import b64encode
import warnings

def from_json(value, obj=None):
  return []

def array_to_binary(ar, obj=None):
  if ar is not None:
    ar = ar.astype(np.int32)
    # ar = ar.astype(np.float32)
    mv = memoryview(ar)
    # 'nested' is just to show the (de)serialization goes all fine
    return {'data': mv, 'shape': ar.shape}
  else:
    return None

last_value = None
setters = 0

def binary_to_array(value, obj=None):
    global last_value, setters
    setters += 1
    #print(">>", value) # print msg'es get lost, but check the websocket output
    last_value = value # or keep a reference to a global for debugging
    return np.frombuffer(value['data'], dtype=np.int32)
    # return np.frombuffer(value['data'], dtype=np.float32)

def array_to_list(ar, obj=None):
    if ar is None:
        return None
    return ar.tolist()

def list_to_array(list, obj=None):
    return np.array(list, dtype=np.float32)

array_binary_serialization = dict(to_json=array_to_binary, from_json=binary_to_array)

array_json_serialization = dict(to_json=array_to_list, from_json=list_to_array)

In [ ]:
default_array = np.array(
    [
        [ 0, 0, 0, 0, 0, ],
        [ 0, 0, 1, 0, 0, ],
        [ 0, 1, 1, 1, 0, ],
        [ 0, 0, 1, 0, 0, ],
        [ 0, 1, 1, 1, 0, ],
        [ 0, 0, 1, 0, 0, ],
        [ 0, 0, 0, 0, 0, ],
    ],
)

@register
class IpyPixelDrawWidget(widgets.DOMWidget):
    _view_name = Unicode("IpyPixelDrawView").tag(sync=True)
    _view_module = Unicode("ipyPixelDraw").tag(sync=True)
    _view_module_version = Unicode("0.0.0").tag(sync=True)

    # Attributes
    canvas_data = Array(
        default_value=default_array,
        allow_none=True,
        help="Array holding canvas content.",
    ).tag( sync=True, **array_binary_serialization, )

    pix_zoom = Integer(
        default_value=36,
        allow_none=False,
        help="Pixel zoom coefficient.",
        ).tag(sync=True)

    @validate("pix_zoom")
    def _valid_pix_zoom(self, proposal):
        if proposal["value"] <= 8:
            raise TraitError("Please ensure that 8 <= pix_zoom < 128")
        if proposal["value"] > 128:
            raise TraitError("Please ensure that 8 <= pix_zoom < 128")
        return proposal["value"]

In [ ]:
%%javascript
require.undef("ipyPixelDraw")

define("ipyPixelDraw", ["@jupyter-widgets/base", ], function(widgets) {

    function deserialize_numpy_array(data, manager) {
        if(data == null)
            return null;
        console.log("binary array")
        window.last_data = data
        // var ar = new Float32Array(data.data.buffer)
        var ar = new Int32Array(data.data.buffer)
        window.last_array = ar
        return {data:ar, shape:data.shape, nested:data.nested}
    }

    function serialize_numpy_array(data, m) {
        console.log("serialize")
        return data;//[0,9]
    }


    var ipyPixelDraw_view = widgets.DOMWidgetView.extend(
        {
            render: function() {

                this.serializeCanvasData = function() {
                    // Int32Array.from( output_flat )
                    var output_flat = this.canvas_data_dictionary
                    var output_flat = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                    // arrayFlattenRecursive
                    function arrayFlattenRecursive( array_rough ) {
                        return array_rough.reduce((acc, val) => Array.isArray(val) ? acc.concat(arrayFlattenRecursive(val)) : acc.concat(val), []);
                    }
                    output_flat = arrayFlattenRecursive( output_flat );
                    this.canvas_data_serialized = { data: new DataView( (new Int32Array( output_flat ) ).buffer ), shape: [7,5] }
                    // this.canvas_data_serialized = { data: new Int32Array( output_flat ).buffer, shape: [7,5] }
                    // { dtype: 'int32' }
                    // value = {buffer: new Float32Array([1,2,3]).buffer, shape: [3]}

                    console.log("+++++++++++++++++++++++++++++++++++++++++++++");
                    console.log("Running: this.serializeCanvasData");
                    console.log("+++++++++++++++++++++++++++++++++++++++++++++");
                    console.log("this.canvas_data_serialized");
                    console.log( this.canvas_data_serialized );
                    console.log("+++++++++++++++++++++++++++++++++++++++++++++");
                    console.log("this.canvas_data_dictionary");
                    console.log( this.canvas_data_dictionary );

                    this.model.set("canvas_data", this.canvas_data_serialized );
                    this.model.save_changes();

                    // this.set("canvas_data", this.canvas_data_serialized );
                    // this.save_changes();

                    // this.trigger("data_updated");
                }

                this.parseCanvasData = function() {
                    console.log("=============================================");
                    console.log("RUNNING: this.parseCanvasData");
                    console.log("=============================================");
                    this.canvas_data_dictionary = this.model.get("canvas_data");
                    console.log("this.canvas_data_dictionary");
                    console.log( this.canvas_data_dictionary );

                    this.canvas_data_shapeArray = this.canvas_data_dictionary.shape;
                    console.log("this.canvas_data_shapeArray");
                    console.log( this.canvas_data_shapeArray );

                    this.canvas_data_typedArray = new Int32Array(this.canvas_data_dictionary.data.buffer); // Checkup on Uint32Array
                    console.log("this.canvas_data_typedArray");
                    console.log( this.canvas_data_typedArray );

                    this.canvas_data_basicArray = Array.from(this.canvas_data_typedArray);
                    console.log("this.canvas_data_basicArray");
                    console.log( this.canvas_data_basicArray );

                    this.canvas_data_rows_count = this.canvas_data_dictionary.shape[0];
                    console.log("this.canvas_data_rows_count");
                    console.log( this.canvas_data_rows_count );

                    this.canvas_data_cols_count = this.canvas_data_dictionary.shape[1];
                    console.log("this.canvas_data_cols_count");
                    console.log( this.canvas_data_cols_count );

                    if ( !Array.isArray( this.canvas_data_basicArray ) ) {
                        throw new Error( "parseCanvasData: argument `this.canvas_data_basicArray` should be an array" );
                    }
                    if ( this.canvas_data_basicArray.length === 0 ) {
                        throw new Error( "parseCanvasData: argument `this.canvas_data_basicArray` should be a non-empty array" );
                    }
                    if ( !Array.isArray( this.canvas_data_shapeArray ) ) {
                        throw new Error( "parseCanvasData: argument `this.canvas_data_shapeArray` should be an array" );
                    }
                    if ( this.canvas_data_shapeArray.length === 0 ) {
                        throw new Error( "parseCanvasData: argument `this.canvas_data_shapeArray` should be a non-empty array" );
                    }
                    if ( this.canvas_data_shapeArray.some( x => Array.isArray(x) ) ) {
                        throw new Error( "parseCanvasData: argument `this.canvas_data_shapeArray` should be a flat (1d) array" );
                    }
                    if ( !this.canvas_data_shapeArray.every( x => ( Number.isInteger(x) ) && ( x > 0 ) ) ) {
                        throw new Error( "parseCanvasData: argument argument `this.canvas_data_shapeArray` should contain only positive (>0) integers" );
                    }
                    // arrayFlattenRecursive
                    function arrayFlattenRecursive( array_rough ) {
                        return array_rough.reduce((acc, val) => Array.isArray(val) ? acc.concat(arrayFlattenRecursive(val)) : acc.concat(val), []);
                    }
                    this.canvas_data_basicArray = arrayFlattenRecursive( this.canvas_data_basicArray );
                    if ( this.canvas_data_basicArray.length !== this.canvas_data_shapeArray.reduce( (a,b) => a * b ) ) {
                        throw new Error( "parseCanvasData: the length of `this.canvas_data_basicArray` should be equal to the product of the elements of `this.canvas_data_shapeArray`" );
                    }
                    // arrayChunkify
                    function arrayChunkify( array_in,  chunk_len ) {
                        var array_out = [];
                        while(array_in.length) {
                            array_out.push(array_in.splice(0, chunk_len));
                        }
                        return array_out;
                    }
                    // arrayReshapeRecursive
                    function arrayReshapeRecursive( array_old,  array_shape ) {
                        var n = 0
                        array_shape = array_shape.slice();
                        if ( array_shape.length > 1 ) {
                            n = array_shape.pop();
                            for ( var ii = 0; ii < array_old.length; ii++ ) {
                                var array_reshaped = arrayChunkify( array_old, n );
                                return arrayReshapeRecursive( array_reshaped, array_shape );
                            }
                        }
                        else {
                            return array_old;
                        }
                    }
                    this.canvas_data = arrayReshapeRecursive( this.canvas_data_basicArray, this.canvas_data_shapeArray );
                }


                this.writeCanvasContentToHTML = function() {
                    console.log("=============================================");
                    console.log("RUNNING: this.writeCanvasContentToHTML");
                    console.log("=============================================");
                    var array_html = "<pre>np.array( [ <br />"
                    for ( var ii = 0; ii < this.canvas_data.length; ii++ ) {
                        array_html = array_html + "    [ ";
                        for ( var jj = 0; jj < this.canvas_data[ii].length; jj++ ) {
                            if (this.canvas_data[ii][jj] == 0) {
                                array_html = array_html + "<span style=\"color: blue; \">";
                                array_html = array_html + this.canvas_data[ii][jj];
                                array_html = array_html + "</span>";
                                array_html = array_html + ", ";
                            }
                            else {
                                array_html = array_html + "<span style=\"color: red; \">";
                                array_html = array_html + this.canvas_data[ii][jj];
                                array_html = array_html + "</span>";
                                array_html = array_html + ", ";
                            }
                        }
                        array_html = array_html + "],<br />";
                    }
                    array_html = array_html + "], )</pre>";
                    this.sketch.innerHTML = array_html;
                }


                this.writeCanvasContentToConsole = function() {
                    console.log("=============================================");
                    console.log("RUNNING: this.writeCanvasContentToConsole");
                    console.log("=============================================");
                    console.log("[");
                    var array_row_str = "";
                    for ( var ii = 0; ii < this.canvas_data.length; ii++ ) {
                        array_row_str = "  [";
                        for ( var jj = 0; jj < this.canvas_data[ii].length; jj++ ) {
                            array_row_str = array_row_str + " " +this.canvas_data[ii][jj];
                            if  ( jj < this.canvas_data[ii].length - 1 ) {
                                array_row_str = array_row_str  + ",";
                            }
                        }
                        array_row_str = array_row_str + " ], // " + ii;
                        console.log(array_row_str);
                    }
                    console.log("]");
                }


                this.drawCanvasContent = function() {
                    console.log("=============================================");
                    console.log("RUNNING: this.drawCanvasContent");
                    console.log("=============================================");
                    for ( var ii = 0; ii < this.canvas_data.length; ii++ ) {
                        for ( var jj = 0; jj < this.canvas_data[ii].length; jj++ ) {
                            if ( this.canvas_data[ii][jj] == 0 ) {
                                this.contxt.fillStyle = this.col_back;
                            }
                            else {
                                this.contxt.fillStyle = this.col_fore;
                            }
                            this.contxt.fillRect( jj*this.pix_zoom, ii*this.pix_zoom, this.pix_zoom, this.pix_zoom );
                        }
                    }
                    // grid: vertical
                    for (var xx = 0; xx <= this.canvas_w; xx += this.pix_zoom) {
                        this.contxt.moveTo( 0.5 + xx + this.pix_zoom , 0             );
                        this.contxt.lineTo( 0.5 + xx + this.pix_zoom , this.canvas_h );
                    }
                    // grid: horizontal
                    for (var yy = 0; yy <= this.canvas_h; yy += this.pix_zoom) {
                        this.contxt.moveTo( 0             , 0.5 + yy + this.pix_zoom );
                        this.contxt.lineTo( this.canvas_w , 0.5 + yy + this.pix_zoom );
                    }
                    this.contxt.strokeStyle = this.col_grid;
                    this.contxt.stroke();
                }


                this.renewCanvasDimensions = function() {
                    console.log("=============================================");
                    console.log("RUNNING: this.renewCanvasDimensions");
                    console.log("=============================================");
                    this.canvas_h = this.canvas_data_rows_count * this.pix_zoom;
                    this.canvas_w = this.canvas_data_cols_count * this.pix_zoom;

                    console.log("this.canvas_data_rows_count");
                    console.log( this.canvas_data_rows_count );
                    console.log("this.canvas_data_cols_count");
                    console.log( this.canvas_data_cols_count );
                    console.log("------------------------------");
                    console.log("this.canvas_w");
                    console.log( this.canvas_w );
                    console.log("this.canvas_h");
                    console.log( this.canvas_h );
                    console.log("------------------------------");

                }


                this.clearAndReDrawCanvas = function() {
                    console.log("=============================================");
                    console.log("RUNNING: this.clearAndReDrawCanvas");
                    console.log("=============================================");

                    this.canvas.width  = this.canvas_w;
                    this.canvas.height = this.canvas_h;

                    // this.contxt = this.canvas.getContext('2d');

                    this.contxt.clearRect( 0, 0, this.canvas.width, this.canvas.height );
                    this.drawCanvasContent()
                }


                this.drawPointerOnCanvas = function(current_pix) {
                    this.contxt.fillStyle = this.col_curs;
                    var pointer_size = Math.ceil( this.pix_zoom/4 )
                    this.contxt.fillRect( current_pix.x-pointer_size, current_pix.y-pointer_size, pointer_size, pointer_size );
                }


                this.getCanvasOrigPixel = function(ev) {
                    let x = ev.clientX;
                    let y = ev.clientY;
                    var bound = this.canvas.getBoundingClientRect();
                    return {x: x-bound.left, y: y-bound.top};
                }

                this.getCanvasZoomPixel = function(e) {
                    let coords = this.getCanvasOrigPixel(e);
                    var current_X_pos = Math.floor( (coords.x-1) / this.pix_zoom );
                    var current_Y_pos = Math.floor( (coords.y-1) / this.pix_zoom );
                    if (current_X_pos > this.canvas_data_cols_count -1) {
                        current_X_pos = this.canvas_data_cols_count -1;
                        console.log( "Correction: X too high!" );
                    }
                    if (current_Y_pos > this.canvas_data_rows_count -1) {
                        current_Y_pos = this.canvas_data_rows_count -1;
                        console.log( "Correction: Y too high!" );
                    }
                    if (current_X_pos < 0) {
                        current_X_pos = 0;
                        console.log( "Correction: X too low!" );
                    }
                    if (current_Y_pos < 0) {
                        current_Y_pos = 0;
                        console.log( "Correction: Y too low!" );
                    }
                    var current_value = this.canvas_data[current_Y_pos][current_X_pos];
                    // console.log( 'X=' + current_X_pos + '; Y=' + current_Y_pos + '; V=' + current_value );
                    return {X: current_X_pos, Y: current_Y_pos, V: current_value};
                }


                this.handleMouseDn = function(ev) {
                    console.log("Running: this.handleMouseDn")
                    this.is_mouse_down = true;
                    let current_mat = this.getCanvasZoomPixel(ev);
                    let current_pix = this.getCanvasOrigPixel(ev);
                    if ( current_mat.V == 0 ) {
                        this.updated_value = 1;
                    }
                    else {
                        this.updated_value = 0;
                    }
                    this.canvas_data[current_mat.Y][current_mat.X] = this.updated_value;
                    this.writeCanvasContentToHTML();
                    this.drawCanvasContent();
                    this.drawPointerOnCanvas(current_pix);
                }

                this.handleMouseMv = function(ev) {
                    console.log("Running: this.handleMouseMv")
                    if (!this.is_mouse_down) { return; }
                    let current_mat = this.getCanvasZoomPixel(ev);
                    let current_pix = this.getCanvasOrigPixel(ev);
                    this.canvas_data[current_mat.Y][current_mat.X] = this.updated_value;
                    this.writeCanvasContentToHTML();
                    this.drawCanvasContent();
                    this.drawPointerOnCanvas(current_pix);
                }


                this.handleMouseUp = function(ev) {
                    console.log("Running: this.handleMouseUp")
                    this.is_mouse_down = false;
                    this.writeCanvasContentToConsole();
                    this.writeCanvasContentToHTML();
                    this.drawCanvasContent();
                }


                this.parseCanvasData();
                this.pix_zoom = this.model.get("pix_zoom");
                this.renewCanvasDimensions();
                this.col_fore = "#bbef53";
                this.col_back = "#663399";
                this.col_grid = "#000000";
                this.col_bord = "#000000";
                this.col_curs = "#ff0066";

                console.log("==============================");
                console.log("DOM");
                console.log("==============================");

                this.canvas = document.createElement("canvas");
                this.canvas.style  = "border: 1px solid " + this.col_bord + "; border-radius: 2px; margin: 0px 16px 16px 0px; float: left; ";
                this.canvas.width  = this.canvas_w;
                this.canvas.height = this.canvas_h;
                this.contxt = this.canvas.getContext('2d');

                this.sketch = document.createElement("div");
                this.sketch.setAttribute( "class", "pixel-draw-sketch" );
                this.sketch.style = "background: #f3f3f3; border: 1px solid #cccccc; border-radius: 2px; padding: 16px 16px 16px 16px; margin: 0px 16px 16px 0px; float: left; ";
                this.sketch.innerHTML = "OK";

                this.block_controls = document.createElement("div");
                this.block_controls = document.createElement("div");
                this.block_controls.setAttribute("class", "pixel-draw-controls");
                this.block_controls.style = "background: #f3f3f3; border: 1px solid red; border-radius: 2px; padding: 16px 16px 16px 16px; margin: 0px 16px 16px 0px; float: left; ";

                this.block_pix_zoom = document.createElement("div");
                this.block_pix_zoom.style = "display: flex; align-items: center;";

                this.label_pix_zoom = document.createElement("label");
                this.label_pix_zoom.innerHTML = "<pre>pix_zoom: </pre>";

                this.input_pix_zoom          = document.createElement("input");
                this.input_pix_zoom.type     = "number";
                this.input_pix_zoom.value    = this.model.get("pix_zoom");
                this.input_pix_zoom.min      = 8;
                this.input_pix_zoom.max      = 128;
                this.input_pix_zoom.step     = 1;
                this.input_pix_zoom.style    = "width: 96px;";
                this.input_pix_zoom.disabled = false;

                this.block_pix_zoom.appendChild(this.label_pix_zoom);
                this.block_pix_zoom.appendChild(this.input_pix_zoom);
                this.block_controls.appendChild(this.block_pix_zoom);


                this.block_btn_push = document.createElement("div");
                this.block_btn_push.style = "display: flex; align-items: center;";

                this.label_btn_push = document.createElement("label");
                this.label_btn_push.innerHTML = "<pre>btn_push: </pre>";

                this.input_btn_push          = document.createElement("input");
                this.input_btn_push.type     = "button";
                this.input_btn_push.value    = "push > py";
                this.input_pix_zoom.style    = "width: 96px;";
                this.input_btn_push.disabled = false;

                this.block_btn_push.appendChild(this.label_btn_push);
                this.block_btn_push.appendChild(this.input_btn_push);
                this.block_controls.appendChild(this.block_btn_push);





                this.writeCanvasContentToConsole();
                this.writeCanvasContentToHTML();
                this.drawCanvasContent();


                this.updated_value = 0
                this.is_mouse_down = false;


                this.canvas.addEventListener('mousedown', ev => this.handleMouseDn(ev), false );
                this.canvas.addEventListener('mousemove', ev => this.handleMouseMv(ev), false );
                this.canvas.addEventListener('mouseup',   ev => this.handleMouseUp(ev), false );


                // ---------------------------------------------------------------------------
                // Append DOM ellements
                // ---------------------------------------------------------------------------

                this.el.append(this.canvas);
                this.el.append(this.sketch);
                this.el.append(this.block_controls);


                // ---------------------------------------------------------------------------
                // Python -> JavaScript update
                // ---------------------------------------------------------------------------

                this.model.on("change:pix_zoom",    this.pix_zoom_changed_from_py,    this);
                this.model.on("change:canvas_data", this.canvas_data_changed_from_py, this);


                // ---------------------------------------------------------------------------
                // JavaScript -> Python update
                // ---------------------------------------------------------------------------

                this.input_pix_zoom.onchange = this.pix_zoom_changed_from_js.bind(this);
                this.input_btn_push.onclick  = this.btn_push_clicked_from_js.bind(this);

            },

            // render ends here


            pix_zoom_changed_from_py: function() {
                console.log("=============================================");
                console.log("Running: pix_zoom_changed_from_py");
                console.log("---------------------------------------------");
                console.log("Before get from model: " + this.model.get("pix_zoom") );
                console.log("Before get from input: " + this.input_pix_zoom.value );
                this.input_pix_zoom.value = this.model.get("pix_zoom");
                this.pix_zoom             = this.model.get("pix_zoom");
                console.log("---------------------------------------------");
                console.log("After get from model: " + this.model.get("pix_zoom") );
                console.log("After get from input: " + this.input_pix_zoom.value );
                this.renewCanvasDimensions();
                this.writeCanvasContentToConsole();
                this.writeCanvasContentToHTML();
                this.clearAndReDrawCanvas();
                this.drawCanvasContent();
            },


            canvas_data_changed_from_py: function() {
                console.log("=============================================");
                console.log("Running: canvas_data_changed_from_py");
                console.log("---------------------------------------------");
                // console.log("Before get from model: " + this.model.get("pix_zoom") );
                // console.log("Before get from input: " + this.input_pix_zoom.value );
                this.parseCanvasData();
                this.renewCanvasDimensions();
                this.writeCanvasContentToConsole();
                this.writeCanvasContentToHTML();
                this.clearAndReDrawCanvas();
                this.drawCanvasContent();
                // console.log("After get from model: " + this.model.get("pix_zoom") );
                // console.log("After get from input: " + this.input_pix_zoom.value );
            },


            pix_zoom_changed_from_js: function() {
                console.log("=============================================");
                console.log("Running: pix_zoom_changed_from_js");
                console.log("---------------------------------------------");
                this.model.set("pix_zoom", parseInt( this.input_pix_zoom.value, 10 ));
                this.model.save_changes();
                // this.renewCanvasDimensions();
                // this.drawCanvasContent();
                // this.canvas_data_changed_from_js.bind(this);
            },


            btn_push_clicked_from_js: function() {
                console.log("=============================================");
                console.log("Running: btn_push_clicked_from_js");
                console.log("---------------------------------------------");
                this.serializeCanvasData();
                // this.model.set("canvas_data", this.canvas_data_serialized );
                // this.model.save_changes();
            },

            canvas_data_changed_from_js: function() {
                console.log("+++++++++++++++++++++++++++++++++++++++++++++");
                console.log("+++++++++++++++++++++++++++++++++++++++++++++");
                console.log("+++++++++++++++++++++++++++++++++++++++++++++");
                console.log("Running: canvas_data_changed_from_js");
                console.log("+++++++++++++++++++++++++++++++++++++++++++++");
                console.log("+++++++++++++++++++++++++++++++++++++++++++++");
                console.log("+++++++++++++++++++++++++++++++++++++++++++++");

                this.model.set("canvas_data", parseInt( this.input_pix_zoom.value, 10 ));

                this.serializeCanvasData();
                this.model.set("canvas_data", this.canvas_data_serialized );
                this.model.save_changes();

                // this.renewCanvasDimensions();
                // this.drawCanvasContent();

            },



        },

        {
            serializers: _.extend({
                canvas_data: {
                    deserialize: deserialize_numpy_array,
                    serialize:     serialize_numpy_array },
            }, widgets.WidgetModel.serializers ) // widgets.WidgetModel.serializers // BaseModel.serializers
        }


    );


    return {IpyPixelDrawView: ipyPixelDraw_view};

})

In [ ]:
pix_1 = IpyPixelDrawWidget(  )

In [ ]:
pix_1

In [ ]:
pix_1.pix_zoom

In [ ]:
pix_1.pix_zoom = pix_1.pix_zoom + 16

In [ ]:
pix_1.pix_zoom = pix_1.pix_zoom - 16

In [ ]:
pix_1.canvas_data

In [ ]:
pix_1.canvas_data = np.array( [
    [ 0, 0, 0, 0, 0, ],
    [ 0, 1, 1, 1, 0, ],
    [ 0, 1, 0, 1, 0, ],
    [ 0, 1, 0, 1, 0, ],
    [ 0, 1, 0, 1, 0, ],
    [ 0, 1, 1, 1, 0, ],
    [ 0, 0, 0, 0, 0, ],
], )

In [ ]:
pix_1.canvas_data

In [ ]:
pix_1.canvas_data = np.array( [
    [ 1, 0, 1, 0, 1, ],
    [ 0, 1, 0, 1, 0, ],
    [ 1, 0, 1, 0, 1, ],
    [ 0, 1, 0, 1, 0, ],
    [ 1, 0, 1, 0, 1, ],
    [ 0, 1, 0, 1, 0, ],
    [ 1, 0, 1, 0, 1, ],
], )

In [ ]:
pix_1